In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
files = os.listdir('./site_data_interpolate')

In [3]:
start = time.time()
for file in files:
    df = pd.read_csv('site_data_interpolate/'+file)
    df.drop(['AQI_bucket'], inplace=True,axis=1)
    df['From Date'] = pd.to_datetime(df['From Date'], format='%Y-%m-%d %H:%M')
    new_df = df.groupby([df['From Date'].dt.date , df['From Date'].dt.hour // 6])[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO', 'Ozone', 'AQI']].agg(['min', 'max', 'mean'])
    new_df.columns = new_df.columns.map('_'.join)
    new_df.index = new_df.index.set_names(['Date', 'Time'])
    new_df.reset_index(inplace=True)
    daywise_df = df.groupby(df['From Date'].dt.date)[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO', 'Ozone', 'AQI']].agg(['min', 'max', 'mean'])
    daywise_df.columns = daywise_df.columns.map('_'.join)
    daywise_df.index.set_names('Date', inplace=True)
    daywise_df.reset_index(inplace=True)
    daywise_df['Time'] = -1
    new_df = pd.concat([new_df, daywise_df])
    new_df.sort_values(['Date', 'Time'], inplace=True)
    new_df['Time'] = new_df['Time'].replace({-1: 'All Day', 0: 'Night', 1: 'Morning', 2: 'Afternoon', 3: 'Evening'})
    new_df.reset_index(drop=True, inplace=True)
    new_df.to_csv('site_aggregate_data_daily/' + file, index=False)
print((time.time() - start) / 60)

4.397366078694661


In [ ]:
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

In [ ]:
for file in files:

    df = pd.read_csv('site_data/' + file)
    new_cols = list(df.columns[2:-1])
    new_df = pd.DataFrame(columns = ['Date', 'Time'] + new_cols)
    time_list = ['night', 'morning', 'afternoon', 'evening']
    k = 0
    for i in range(0, len(df), 6):
        df1 = df.iloc[i:i+6]
        curr_date = df1.loc[df1.index[0], 'From Date'].split(' ')[0]
        curr_time = time_list[k]
        k = (k+1) % 4
        average_list = []
        average_row = df1.iloc[:,2:-1].mean(skipna=True)

        new_row = pd.concat([pd.Series({'Date': curr_date, 'Time': curr_time}), average_row])

        new_df.loc[len(new_df.index)] = new_row

        if k == 0:
            average_row = new_df.iloc[-4:,2:].mean(skipna=True)
            new_row = pd.concat([pd.Series({'Date': curr_date, 'Time': 'all_day'}), average_row])
            new_df.loc[len(new_df.index)] = new_row
    new_df["AQI_bucket"] = new_df["AQI"].apply(lambda x: get_AQI_bucket(x))
    new_df.to_csv('site_aggregate_data_daily/' + file, index=False)

In [ ]:
new_df = pd.DataFrame(columns = ['Date', 'Time'] + new_cols)

In [ ]:
col = pd.Series({'Date': curr_date, 'Time': curr_time})
col

In [ ]:
average_row.shape

In [ ]:
new_df

In [ ]:
new_df

In [ ]:
pd.concat([new_df, avg_row], axis=0, ignore_index=True)

In [ ]:
new_df

In [ ]:
import numpy as np

In [ ]:
a = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

In [ ]:
a[-2:]